<a href="https://colab.research.google.com/github/74obunga/-image-classifier-using-the-VGG16-pre-trained-model/blob/main/DL0321EN_4_1_Comparing_Models_py_v1_0_(4).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [1]:
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip -O concrete_data_week4.zip


--2024-07-25 17:16:22--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261483817 (249M) [application/zip]
Saving to: ‘concrete_data_week4.zip’

concrete_data_week4 100%[===================>] 249.37M  10.2MB/s    in 24s     

2024-07-25 17:16:47 (10.3 MB/s) - ‘concrete_data_week4.zip’ saved [261483817/261483817]



In [2]:
import zipfile
import os

# Ensure the zip file is correctly downloaded
if os.path.exists('concrete_data_week4.zip'):
    with zipfile.ZipFile('concrete_data_week4.zip', 'r') as zip_ref:
        zip_ref.extractall('concrete_data_week4')
else:
    print("Download failed or the file doesn't exist.")



In [3]:
import os

# Check the contents of the main directory
print(os.listdir('concrete_data_week4'))

# Check the contents of any subdirectories
for subdir in os.listdir('concrete_data_week4'):
    print(f"Contents of {subdir}:")
    print(os.listdir(os.path.join('concrete_data_week4', subdir)))


['__MACOSX', 'concrete_data_week4']
Contents of __MACOSX:
['concrete_data_week4']
Contents of concrete_data_week4:
['.DS_Store', 'valid', 'train', 'test']


After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

In [4]:
import os

# Check the contents of the main directory
main_dir = 'concrete_data_week4'
print(f"Contents of '{main_dir}':", os.listdir(main_dir))

# Check the subdirectories inside the main directory
for subdir in os.listdir(main_dir):
    subdir_path = os.path.join(main_dir, subdir)
    if os.path.isdir(subdir_path):
        print(f"Contents of '{subdir_path}':", os.listdir(subdir_path))


Contents of 'concrete_data_week4': ['__MACOSX', 'concrete_data_week4']
Contents of 'concrete_data_week4/__MACOSX': ['concrete_data_week4']
Contents of 'concrete_data_week4/concrete_data_week4': ['.DS_Store', 'valid', 'train', 'test']


<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [12]:
from tensorflow.keras.applications import ResNet50

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

batch_size = 100

# Define ImageDataGenerators
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Correct paths based on the actual structure
train_generator = train_datagen.flow_from_directory(
    'concrete_data_week4/concrete_data_week4/train',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_directory(
    'concrete_data_week4/concrete_data_week4/valid',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

# Load the VGG16 model
vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Create a new model on top
model = Sequential()
model.add(vgg16_base)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(2, activation='softmax'))  # Adjust number of classes as needed

# Freeze the base VGG16 layers
vgg16_base.trainable = False

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=10
)




Found 30001 images belonging to 2 classes.
Found 9501 images belonging to 2 classes.
58889256/58889256 [==============================] - 0s 0us/step
Epoch 1/10
300/300 [==============================] - 454s 1s/step - loss: 0.1019 - accuracy: 0.9895 - val_loss: 0.0135 - val_accuracy: 0.9976
Epoch 2/10
300/300 [==============================] - 436s 1s/step - loss: 0.0331 - accuracy: 0.9951 - val_loss: 0.0143 - val_accuracy: 0.9962
Epoch 3/10
300/300 [==============================] - 436s 1s/step - loss: 0.0175 - accuracy: 0.9965 - val_loss: 0.0091 - val_accuracy: 0.9966
Epoch 4/10
300/300 [==============================] - 436s 1s/step - loss: 0.0094 - accuracy: 0.9974 - val_loss: 0.0091 - val_accuracy: 0.9972
Epoch 5/10
300/300 [==============================] - 437s 1s/step - loss: 0.0071 - accuracy: 0.9984 - val_loss: 0.0066 - val_accuracy: 0.9980
Epoch 6/10
300/300 [==============================] - 436s 1s/step - loss: 0.0074 - accuracy: 0.9978 - val_loss: 0.0055 - val_accuracy:

In [13]:
# Save the trained VGG16 model
model.save('vgg16_model.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
#

SyntaxError: unmatched ')' (<ipython-input-9-35f86578a457>, line 40)

In [16]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

# Define paths
train_data_dir = 'concrete_data_week4/concrete_data_week4/train'
valid_data_dir = 'concrete_data_week4/concrete_data_week4/valid'
test_data_dir = 'concrete_data_week4/concrete_data_week4/test'

# ImageDataGenerators
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_directory(
    valid_data_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

# Load ResNet50 base model
resnet50_base = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Create a new model on top
resnet50_model = Sequential()
resnet50_model.add(resnet50_base)
resnet50_model.add(Flatten())
resnet50_model.add(Dense(256, activation='relu'))
resnet50_model.add(Dense(2, activation='softmax'))  # Adjust number of classes as needed

# Freeze the base ResNet50 layers
resnet50_base.trainable = False

# Compile the model
resnet50_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
resnet50_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=10
)

# Save the trained model
resnet50_model.save('resnet50_model.h5')


Found 30001 images belonging to 2 classes.
Found 9501 images belonging to 2 classes.
94765736/94765736 [==============================] - 0s 0us/step
Epoch 1/10
300/300 [==============================] - 143s 446ms/step - loss: 0.4330 - accuracy: 0.9913 - val_loss: 0.0092 - val_accuracy: 0.9966
Epoch 2/10
300/300 [==============================] - 132s 438ms/step - loss: 0.0023 - accuracy: 0.9996 - val_loss: 0.0070 - val_accuracy: 0.9983
Epoch 3/10
300/300 [==============================] - 132s 439ms/step - loss: 5.6850e-04 - accuracy: 0.9999 - val_loss: 0.0067 - val_accuracy: 0.9983
Epoch 4/10
300/300 [==============================] - 131s 438ms/step - loss: 2.8088e-04 - accuracy: 0.9998 - val_loss: 0.0070 - val_accuracy: 0.9986
Epoch 5/10
300/300 [==============================] - 131s 436ms/step - loss: 1.6286e-04 - accuracy: 0.9999 - val_loss: 0.0071 - val_accuracy: 0.9983
Epoch 6/10
300/300 [==============================] - 132s 440ms/step - loss: 1.3128e-04 - accuracy: 0.9999 

<a id="item43"></a>

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model.
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [17]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the path to the saved model
resnet50_model_path = 'resnet50_model.h5'

# Load the saved ResNet50 model
resnet50_model = load_model(resnet50_model_path)

# Create an ImageDataGenerator for the test set
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Evaluate the ResNet50 model
resnet50_loss, resnet50_accuracy = resnet50_model.evaluate(test_generator)
print(f"ResNet50 Model - Loss: {resnet50_loss}, Accuracy: {resnet50_accuracy}")

# Evaluate the VGG16 model
vgg16_loss, vgg16_accuracy = model.evaluate(test_generator)  # 'model' should be your trained VGG16 model
print(f"VGG16 Model - Loss: {vgg16_loss}, Accuracy: {vgg16_accuracy}")


Found 500 images belonging to 2 classes.
5/5 [==============================] - 3s 327ms/step - loss: 0.0010 - accuracy: 1.0000
ResNet50 Model - Loss: 0.001017495640553534, Accuracy: 1.0
5/5 [==============================] - 2s 389ms/step - loss: 3.6416e-04 - accuracy: 1.0000
VGG16 Model - Loss: 0.00036415987415239215, Accuracy: 1.0


In [19]:
# Number of images used for training
num_train_images = train_generator.samples
print(f"Number of images used for training the VGG16 model: {num_train_images}")


Number of images used for training the VGG16 model: 30001


In [20]:
# Assuming `model` is your complete model including VGG16 base and added layers
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 256)               6422784   
                                                                 
 dense_1 (Dense)             (None, 2)                 514       
                                                                 
Total params: 21137986 (80.64 MB)
Trainable params: 6423298 (24.50 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li>
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [18]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np

# Define the path to the saved VGG16 model
vgg16_model_path = 'vgg16_model.h5'

# Load the saved VGG16 model
vgg16_model = load_model(vgg16_model_path)

# Define the path to the test data directory
test_data_dir = 'concrete_data_week4/concrete_data_week4/test'

# Create an ImageDataGenerator for the test set
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(224, 224),
    batch_size=1,  # Predicting one image at a time
    class_mode='categorical',
    shuffle=False
)

# Predict the class of the images in the test data
predictions = vgg16_model.predict(test_generator)

# Get the class indices
class_indices = {v: k for k, v in test_generator.class_indices.items()}

# Convert predictions to class labels
predicted_classes = np.argmax(predictions, axis=1)
predicted_labels = [class_indices[idx] for idx in predicted_classes]

# Report the class predictions of the first five images
for i in range(5):
    print(f"Image {i + 1}: {predicted_labels[i]}")


Found 500 images belonging to 2 classes.
500/500 [==============================] - 4s 8ms/step
Image 1: negative
Image 2: negative
Image 3: negative
Image 4: negative
Image 5: negative


### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).